In [1]:
import pandas as pd
import numpy as np
import xlrd, sqlite3, os, sys
from datetime import date

In [2]:
# Import Raw Data from excel files into dataFrames
df10=pd.read_excel('orig_data/mlb_odds_2010.xlsx')
df10['Year'] = 2010
df11=pd.read_excel('orig_data/mlb_odds_2011.xlsx')
df11['Year'] = 2011
df12=pd.read_excel('orig_data/mlb_odds_2012.xlsx')
df12['Year'] = 2012
df13=pd.read_excel('orig_data/mlb_odds_2013.xlsx')
df13['Year'] = 2013
df14=pd.read_excel('orig_data/mlb_odds_2014.xlsx')
df14['Year'] = 2014
df15=pd.read_excel('orig_data/mlb_odds_2015.xlsx')
df15['Year'] = 2015
df16=pd.read_excel('orig_data/mlb_odds_2016.xlsx')
df16['Year'] = 2016
df17=pd.read_excel('orig_data/mlb_odds_2017.xlsx')
df17['Year'] = 2017
df18=pd.read_excel('orig_data/mlb_odds_2018.xlsx')
df18['Year'] = 2018
df19=pd.read_excel('orig_data/mlb_odds_2019.xlsx')
df19['Year'] = 2019

In [3]:
# make list of dataFrames
dfList = [df10, df11, df12, df13, df14, df15, df16, df17, df18, df19]

In [4]:
# reformat data
for df in dfList:
    try:
        dfnew=df['RL'].str.split(pat="(", n=1, expand=True)
        dfnewer = dfnew[1].str.rstrip("\)")
        df['run_line_close'] = dfnew[0].apply(float)
        df['run_line_odds_close'] = dfnewer.apply(int)
    except KeyError:
        pass

In [5]:
for df in dfList:
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
    df.rename({"vh": "visitor_home", "final": "team_run_total", "close":"money_line_close", "open":"money_line_open", "openou":"over_under_line_open", "open_ou":"over_under_line_open", "unnamed:_18":"over_under_odds_open", "unnamed:_19":"over_under_odds_open", "close_ou":"over_under_line_close", "closeou":"over_under_line_close", "unnamed:_20":"over_under_odds_close", "unnamed:_21":"over_under_odds_close"}, axis='columns', inplace=True)    

In [6]:
def convt(num):
    if type(num) == str:
        num = float(num[0]) + .5
    return num
for df in dfList:
    df['over_under_line_close'] = df['over_under_line_close'].apply(convt)

In [7]:
# create additional columns
for df in dfList:
    totRuns = []
    counter = 0
    try:
        for index, row in df.iterrows():
            if row.team_run_total == "NL":
                totRuns.append(np.nan)
            elif index % 2 == 0:
                counter = 0
                counter += row.team_run_total
            else:
                counter += row.team_run_total
                totRuns.append(counter)
                totRuns.append(counter)
    except TypeError:
        print(row)
    df['total_runs_game'] = totRuns

In [8]:
for df in dfList:
    RunDif = []
    counter = 0
    try:
        for index, row in df.iterrows():
            if row.team_run_total == "NL":
                RunDif.append(np.nan)
            elif index % 2 == 0:
                counter = 0
                team1 = row.team_run_total
            else:
                team2 = row.team_run_total
                dif = team1 - team2
                RunDif.append(dif)
                RunDif.append(-dif)
    except TypeError:
        print(row)
    df['run_dif_game'] = RunDif

In [9]:
# for df in dfList:
#     winRL = []
#     for index, row in df.iterrows():
#         # favorite, win by more than 1.5 = True
#         # favorite, win by less than 1.5 or lose = False
#         # underdog, wins or loses by less than 1.5 = True
#         # underdog, loses by more than 1.5 = True
#         try:
#             if row.run_line_close < 0:
#                 if row.run_dif_game > abs(row.run_line_close):
#                     winRL.append(1)
#                 else:
#                     winRL.append(-1)
#             else:
#                 if row.run_dif_game > 0 or abs(row.run_dif_game) < row.run_line_close:
#                     winRL.append(1)
#                 else:
#                     winRL.append(-1)
#         except AttributeError:
#             winRL.append(np.nan)
#         except TypeError:
#             winRL.append(np.nan)
#         except ValueError:
#             winRL.append(np.nan)
#     df['run_line_win'] = winRL

In [10]:
# for df in dfList:
#     winTot = []
#     for index, row in df.iterrows():
#         try:
#             if index % 2 == 0:
#                 if row.total_runs_game > row.over_under_line_close:
#                     winTot.append(1)
#                 elif row.total_runs_game < row.over_under_line_close:
#                     winTot.append(-1)
#                 else:
#                     winTot.append(0)
#             else:
#                 if row.total_runs_game > row.over_under_line_close:
#                     winTot.append(-1)
#                 elif row.total_runs_game < row.over_under_line_close:
#                     winTot.append(1)
#                 else:
#                     winTot.append(0)
#         except TypeError:
#             winTot.append(0)
#     df['over_under_win'] = winTot

In [11]:
# for df in dfList:
#     winML = []
#     for index, row in df.iterrows():
#         # favorite, win = True
#         # favorite, lose = False
#         # underdog, wins = True
#         # underdog lose = True
#         try:
#             if row.run_dif_game > 0:
#                 winML.append(1)
#             else:
#                 winML.append(-1)
#         except AttributeError:
#             winML.append(np.nan)
#         except TypeError:
#             winML.append(np.nan)
#         except ValueError:
#             winML.append(np.nan)
#     df['money_line_win'] = winML

In [12]:
# convert date format
for df in dfList:
    df['date'] = df['date'].astype(str) + df['year'].astype(str)
    pd.to_numeric(df['date'])
    df['date'] = pd.to_datetime(df['date'], errors='ignore', format="%m%d%Y")

In [13]:
df1 = pd.concat(dfList, sort=True)

In [14]:
df1.drop(['year', 'rl', '1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th'], axis=1, inplace=True)

In [15]:
colNew = []
i = 0
for index, row in df1.iterrows():
    if index == 0:
        pass
    elif index % 2 == 0:
        i += 1
    colNew.append(i)
df1['gameno'] = colNew
df1.replace('LOS','LAD',inplace=True)
df1.replace('CUB','CHC',inplace=True)
df1.replace('SDG','SD',inplace=True)
df1.replace('SFO','SF',inplace=True)
df1.replace('KAN','KC',inplace=True)
df1.replace('TAM','TB',inplace=True)

In [16]:
df1 = df1.reindex(columns=['date', 'gameno', 'team', 'visitor_home', 'team_run_total', 'total_runs_game', 'money_line_close', 'money_line_open', 'over_under_line_close', 'over_under_line_open', 'over_under_odds_close', 'over_under_odds_open', 'pitcher', 'rot', 'run_dif_game', 'run_line_close', 'run_line_odds_close'])

In [17]:
df1 = df1[df1.over_under_line_close < 100]

In [18]:
df1 = df1[df1.over_under_line_close > 0]

In [19]:
df1['over_under_line_close'].unique()

array([ 9. ,  7.5,  7. ,  8.5,  8. ,  9.5, 10. , 10.5, 11.5,  6.5, 11. ,
       13. , 14.5,  6. , 12. ,  5.5, 12.5, 14. ,  5. , 13.5])

In [20]:
# df1.set_index('date', inplace=True)

In [21]:
df1.replace(['NL', 'nan'], np.nan, inplace=True)

In [22]:
csv_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), 'data', 'baseball.csv'))
df1.to_csv(csv_path)

In [23]:
# team_run_total, run_dif_game has NL, run_line_close & run_line_odds_close needs to change nan to 'NaN'

In [24]:
# DBFILENAME = os.path.join(os.path.abspath(""), 'baseball.db')
# with sqlite3.connect(DBFILENAME) as conn:
#     df1.to_sql('games', con=conn, if_exists='replace', dtype={'date': 'DATETIME', '1st': 'INTEGER', '2nd': 'INTEGER', '3rd': "INTEGER", '4th':'INTEGER', '5th': 'INTEGER', '6th': 'INTEGER', '7th':'INTEGER', '8th': 'INTEGER', '9th': 'INTEGER', 'close': 'REAL', 'close_ou': 'REAL', 'closeouodds': 'REAL', 'final': 'REAL', 'gameno': 'REAL', 'open':"REAL", 'open_ou': 'REAL', 'openou':'REAL', 'openouodds': 'REAL', 'pitcher': 'TEXT', 'rot':'INTEGER', 'rundif': 'REAL', 'runline':'REAL', 'runlineodds': 'REAL', 'team': 'TEXT', 'totalruns': 'INTEGER', 'vh': 'TEXT', 'winrl':'BOOLEAN', 'wintotal': 'BOOLEAN'})